In [55]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the training data
corpus = [
    "I love cats",
    "I adore dogs",
    "Cats are cute",
    "Dogs are loyal"
]

# Create a vocabulary from the corpus
vocab = set(" ".join(corpus).split())
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Convert the corpus to training examples
train_inputs = []
train_targets = []
for sentence in corpus:
    words = sentence.split()
    for i in range(len(words)):
        context = words[:i] + words[i+1:]
        target = words[i]
        train_inputs.append(context)
        train_targets.append(target)


# Define the neural network model
class SimpleLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(SimpleLanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.hidden = nn.Linear(embedding_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, vocab_size)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        embedded = self.embedding(x)
        hidden = self.hidden(embedded)
        output = self.output(hidden)
        probabilities = self.softmax(output)
        return probabilities

# Define model hyperparameters
vocab_size = len(vocab)
embedding_dim = 10
hidden_dim = 20

# Create an instance of the language model
model = SimpleLanguageModel(vocab_size, embedding_dim, hidden_dim)

# Define the loss function
loss_function = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Convert training data to tensor
train_inputs = torch.tensor([[word_to_idx[word] for word in context] for context in train_inputs])
train_targets = torch.tensor([word_to_idx[word] for word in train_targets])

# Repeat each target word twice to match the batch size
train_targets = train_targets.repeat(2)

# Training loop
for epoch in range(100):
    optimizer.zero_grad()

    # Forward pass
    output = model(train_inputs)
    
    # Calculate the loss
    loss = loss_function(output.view(-1, vocab_size), train_targets)
    
    # Backward pass
    loss.backward()
    optimizer.step()

    # Print the loss for every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Example usage
example_input = torch.tensor([word_to_idx[word] for word in ["I", "love"]])
output = model(example_input.unsqueeze(0))
predicted_word_idx = torch.argmax(output)

predicted_word = idx_to_word[predicted_word_idx.item()]
print(f"Next predicted word: {predicted_word}")


Epoch 10, Loss: 2.274981737136841
Epoch 20, Loss: 2.247117757797241
Epoch 30, Loss: 2.2201223373413086
Epoch 40, Loss: 2.1932365894317627
Epoch 50, Loss: 2.1660993099212646
Epoch 60, Loss: 2.138852834701538
Epoch 70, Loss: 2.1121013164520264
Epoch 80, Loss: 2.086674928665161
Epoch 90, Loss: 2.063289165496826
Epoch 100, Loss: 2.042316198348999
Next predicted word: cats
